In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from models import TADW, TriDnr, DeepWalk, Node2Vec, Hope
from text_transformers import SBert, LDA, W2V, Sent2Vec, Doc2Vec, BOW, TFIDF
from datasets import Cora, CiteseerM10, Dblp

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from collections import defaultdict

from task import Task

Using backend: pytorch


In [2]:
datasets = [
   ('Cora', Cora),
   # ('CiteseerM10', CiteseerM10),
   #  ('DBLP', Dblp)
]

test_ratios = [0.5, 0.7, 0.9, 0.95]


tasks = [
    # ('BOW', lambda ds: Task(ds, test_ratios, lambda: BOW(), None, d=None, labels=False)),
    # ('TFIDF', lambda ds: Task(ds, test_ratios, lambda: TFIDF(), None, d=None, labels=False)),
    # ('LDA', lambda ds: Task(ds, test_ratios, lambda: LDA(), None, d=None, labels=False)),
    # ('SBERT pretrained', lambda ds: Task(ds, test_ratios, lambda: SBert(train=False, d=300), None, d=None, labels=False)),
    # ('W2V pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=False, d=300), None, d=None, labels=False)),
    # ('W2V (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=300), None, d=None, labels=False)),
    # ('W2V (d=64)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), None, d=None, labels=False)),
    # ('Doc2Vec pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=False, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('Sent2Vec pretrained (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=False, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('DeepWalk (d=100)', lambda ds: Task(ds, test_ratios, None, DeepWalk, d=100, labels=False)),
    # ('Node2Vec (d=100)', lambda ds: Task(ds, test_ratios, None, Node2Vec, d=100, labels=False)),
    # ('Hope (d=100)', lambda ds: Task(ds, test_ratios, None, Hope, d=100, labels=False)),
    ('TADW - BOW', lambda ds: Task(ds, test_ratios, BOW, TADW, d=160, labels=False)),
    ('TADW - TFIDF', lambda ds: Task(ds, test_ratios, TFIDF, TADW, d=160, labels=False)),
    ('TADW - Sent2Vec', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), TADW, d=160, labels=False)),
    ('TADW - Word2Vec', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), TADW, d=160, labels=False)),
    # ('TriDNR', lambda ds: Task(ds, test_ratios, None, TriDnr, d=160, labels=True)),
    # ('BOW:DeepWalk', lambda ds: Task(ds, test_ratios, BOW, DeepWalk, d=100,
    #                                       labels=False, concat=True)),
    # ('Word2Vec:DeepWalk', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), DeepWalk, d=100,
    #                                       labels=False, concat=True)),
    # ('Sent2Vec:DeepWalk', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), DeepWalk, d=100,
    #                                       labels=False, concat=True)),
]

In [3]:
res = {}

for ds_name, ds_constr in tqdm(datasets, desc='datasets'):
    ds = ds_constr()
    for task_name, task_constr in tqdm(tasks, desc='Tasks'):
        task = task_constr(ds)
        task_res = task.evaluate()
        for test_ratio in task_res:
            scores = task_res[test_ratio]
            res[f'{1 - test_ratio} - {ds_name} - {task_name}'] = scores

for name, scores in res.items():
    print(name, scores, np.mean(scores), np.std(scores))



Tasks:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19




test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  5.63it/s]


seeds:  40%|████      | 2/5 [00:00<00:00,  5.54it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  5.51it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00,  5.64it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00,  5.93it/s]




test_ratios:  25%|██▌       | 1/4 [00:00<00:02,  1.15it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  9.17it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  9.59it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 10.10it/s]




test_ratios:  50%|█████     | 2/4 [00:01<00:01,  1.33it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 21.47it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 20.81it/s]

test_ratios:  75%|███████▌  | 3/4 [00:01<00:00,  1.67it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████   

Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19




test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  8.85it/s]


seeds:  40%|████      | 2/5 [00:00<00:00,  8.78it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  8.91it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00,  8.94it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00,  8.66it/s]




test_ratios:  25%|██▌       | 1/4 [00:00<00:01,  1.74it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  40%|████      | 2/5 [00:00<00:00, 14.04it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00, 14.17it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 14.13it/s]

test_ratios:  50%|█████     | 2/4 [00:00<00:01,  1.96it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 25.07it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 24.70it/s]

test_ratios:  75%|███████▌  | 3/4 [00:01<00:00,  2.39it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 

Traceback (most recent call last):
  File "/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-7e716fa3f95b>", line 7, in <module>
    task_res = task.evaluate()
  File "./task/task.py", line 28, in evaluate
    self.dataset.transform_features(transformer)
  File "./datasets/dataset.py", line 63, in transform_features
    text_embs = transformer.fit_transform(self.texts, **kwargs)
  File "./text_transformers/sent2vec.py", line 151, in fit_transform
    sentence_embeddings = get_sentence_embeddings(texts, self.train, self.d)
  File "./text_transformers/sent2vec.py", line 139, in get_sentence_embeddings
    FASTTEXT_EXEC_PATH, train, d)
  File "./text_transformers/sent2vec.py", line 107, in get_embeddings_for_preprocessed_sentences
    , shell=True
  File "/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/su

KeyboardInterrupt: 